### Imports

In [ ]:
# general imports
import os
import sys
import io
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, Normalizer
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.optimizers import RMSprop

# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

# filepath for project folder
path = '/content/drive/My Drive/RESEARCH/NG_DISCRIMINATION/'

# to play sound when cell finishes executing
from IPython.display import Audio
sound_file1 = path + 'wow.mp3'
sound_file2 = path + 'ding.mp3'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Contents

In [ ]:
def evaluate_NG(model, train_labels, train_data, evaluation_labels, evaluation_data, metaModel, metaCollection):
  print(metaModel, metaCollection)

  #print("Loading training data...")
  td = np.load(train_data)

  #print("Extracting training sequences...")
  train_seqs = []
  for i in range(len(td)):
      train_seqs.append(td[i][11])
  train_seqs = np.array(train_seqs)
  train_seqs = train_seqs.astype('float64')

  #print("Loading training label data...")
  tl = np.load(train_labels)

  #print("Extracting training labels...")
  train_labels = []
  for i in range(len(tl)):
      if tl[i][10] == 1:
          train_labels.append(0)
      elif tl[i][10] == 2:
          train_labels.append(1)
  train_labels = np.array(train_labels)

  #print("Calculating normalization parameters...")
  xtrain, xtest, ytrain, ytest = train_test_split(train_seqs, train_labels, test_size = .2, shuffle = True, random_state = 8)
  normalizer = Normalizer(copy = False).fit(xtrain)

  #print("Loading evaluation data...")
  ed = np.load(evaluation_data)

  #print("Extracting evaluation sequences...")
  eva_seqs = []
  for i in range(len(ed)):
      eva_seqs.append(ed[i][11])
  eva_seqs = np.array(eva_seqs)
  eva_seqs = eva_seqs.astype('float64')

  #print("Normalizing...")
  n_eva_seqs = normalizer.transform(eva_seqs)
  n_eva_seqs = np.reshape(n_eva_seqs, (n_eva_seqs.shape[0], n_eva_seqs.shape[1], 1))

  #print("Loading evaluation label data...")
  el = np.load(evaluation_labels)

  #print("Extracting evaluation labels...")
  eva_labels = []
  for i in range(len(el)):
      if el[i][10] == 1:
          eva_labels.append(0)
      elif el[i][10] == 2:
          eva_labels.append(1)
  eva_labels = np.array(eva_labels)

  #print("Loading model...")
  trained_model = load_model(model)

  #print("Conducting evaluation...")
  results = trained_model.evaluate(x = n_eva_seqs, y = eva_labels, batch_size = 16, verbose = 1)
  
  print("Results:")
  print('test loss, test acc:', results)

### Run Evaluation

In [ ]:
pModel = path + "models/WLS_v2.0.hdf5"
pTrainingLabels = path + "datasets/202002192219/ols.npy" # do not change
pTrainingData = path + "datasets/202002192219/wls.npy" # do not change
pEvalLabels = path + "datasets/202002192219/ols.npy"
pEvalData = path + "datasets/202002192219/wls.npy"
#evaluate_NG(pModel, pTraining, pLabels, pEval)

Models = ['WLS_v1.0.hdf5','WLS_v2.0.hdf5','WLS_v3.0.hdf5']
Collections = ['202002192219','202002200023','202002192353','202002192255','202002200054','202002192326']

for i in range(len(Models)):
  Audio(sound_file1, autoplay=True)
  print('////////////////////////////////////////////////////////////////////////////////////////////')
  for j in range(len(Collections)):
    evaluate_NG(path + 'models/' + Models[i],                       # model
                pTrainingLabels,                                    # training labels (get thrown out)
                pTrainingData,                                      # training data (used to calculate normalization parameters)
                path + 'datasets/' + Collections[j] + '/ols.npy',   # evaluation data
                path + 'datasets/' + Collections[j] + '/wls.npy',   # evaluation labels
                Models[i],                                          # model name
                Collections[j])                                     # collection date
    Audio(sound_file2, autoplay=True)

In [ ]:
evaluate_NG(path + "models/WLS_1F_v1.0.hdf5",
            path + "datasets/202002192353/ols.npy",
            path + "datasets/202002192353/wls.npy",
            path + "datasets/202002192353/ols.npy",
            path + "datasets/202002192353/wls.npy",
            "WLS_1F_v1.0.hdf5",
            "202002192353")

WLS_1F_v1.0.hdf5 202002192353
11141/11141 [==============================] - 78s 7ms/step - loss: 0.3840 - accuracy: 0.8392
Results:
test loss, test acc: [0.3840163052082062, 0.8392022848129272]
